In [ ]:
# 기사 목록(네이버)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib.parse
import time

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 키워드 목록과 날짜 범위 설정
keywords = [
    "검색어",
]
start_date = "2024.06.01"
end_date = "2024.08.31"
start_date = "202040601"
end_date = "20240831"

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

# 모든 결과를 저장할 리스트
all_results = []

# 키워드마다 검색 수행
for keyword in keywords:
    try:
        search_url = "https://search.naver.com/search.naver?where=news&query=" + urllib.parse.quote(keyword) + f"&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=2&office_section_code=8&news_office_checked=1001&nso=so%3Ar%2Cp%3Afrom{start_date}to{start_date}&is_sug_officeid=0&office_category=0&service_area=0"
        driver.get(search_url)

                # 검색 결과 없음 확인 (div.not_found02 선택자 사용)
        no_results_message = driver.find_elements(By.CSS_SELECTOR, "div.not_found02")
        if no_results_message:
            print(f"'{keyword}'에 대한 검색결과가 없습니다.")
            continue  # 검색결과가 없으면 다음 키워드로 넘어감

        SCROLL_PAUSE_TIME = 1

        # 스크롤 끝까지 내리기
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        collected_urls = set()
        article_count = 0
        articles = driver.find_elements(By.CSS_SELECTOR, 'li.bx')

        for article in articles:
            try:
                title_tag = article.find_element(By.CSS_SELECTOR, 'a.news_tit')
                url = title_tag.get_attribute('href')
                title = title_tag.get_attribute('title')
                date_tag = article.find_element(By.CSS_SELECTOR, 'span.info')
                date = date_tag.text.strip()

                if url and url not in collected_urls:
                    collected_urls.add(url)
                    article_count += 1

                    # 결과 리스트에 추가
                    all_results.append({
                        'media': '연합뉴스', 
                        'title': title, 
                        'date': date, 
                        'section': '', 
                        'article_type': '', 
                        'writer': '', 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
                    print(f"기사 추가: {title}, {url}")  # 로그 추가

            except Exception as e:
                print(f"오류 발생: {e}")
                continue

        print(f"키워드 '{keyword}' 검색 완료: 총 {article_count}개 기사")

    except Exception as e:
        print(f"키워드 '{keyword}' 검색 중 오류 발생: {e}")
        continue

driver.quit()

# 결과 출력
df = pd.DataFrame(all_results)
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
print(df)

# 결과를 엑셀 파일로 저장
df.to_excel(f'4연합뉴스 {start_date}~{end_date}.xlsx', index=False)


In [ ]:
# 본문 추출

import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

def wait_for_article_body(wait, class_names):
    """주어진 클래스 이름 중 첫 번째로 매칭된 요소를 반환합니다."""
    for class_name in class_names:
        try:
            return wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
        except TimeoutException:
            continue  # 다음 클래스 이름으로 시도
    return ''

def get_content_from_url(url):
    try:
        driver.get(url)

        # WebDriverWait 설정
        wait = WebDriverWait(driver, 5)

        # 기사 본문을 찾기 위한 클래스 이름 리스트
        class_names = ['article_content', 'newsct_article', 'article-txt', 'story-news', 'content01', 'story-news.article', '_article_content']

        # 첫 번째로 매칭된 요소 찾기
        article_body_element = wait_for_article_body(wait, class_names)

        if article_body_element:
            # 페이지 소스에서 BeautifulSoup로 변환
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 기사 본문 부분만 추출
            article_body_soup = soup.find(class_=class_names)
            
            if article_body_soup:
                # 불필요한 요소를 먼저 제거합니다.
                for tag in article_body_soup.find_all(True):
                    class_attr = tag.get('class', [])
                    if any(cls in class_attr for cls in ['txt-con', 'img_center', 'article-figure', 'nbd_table', 'comp-box', 'article-img', 'img-con', 'chn_box', 'title-ad01', 'title-ad02', 'article-ad-box', 'blind', 'txt-copyright adrs']):
                        tag.extract()  # 태그를 완전히 제거
                        
                # 본문에서 <span data-type="ore"> 태그를 제거하고 안의 텍스트만 남김
                for span_tag in article_body_soup.find_all('span', {'data-type': 'ore'}):
                    span_text = span_tag.get_text(separator='', strip=True)  # 줄바꿈 없이 텍스트만 추출
                    span_tag.replace_with(span_text)
                                # 서브타이틀 추출
                subtitle = ""
                subtitle_tag = soup.find('div', class_='tit-sub')
                if subtitle_tag:
                    subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                    subtitle_tag.extract()  # 서브 타이틀 제거             

                # 본문 텍스트 추출
                content = article_body_soup.get_text(separator='\n', strip=True)  # 텍스트 추출

                # 본문에서 "yna.co.kr" 이후의 모든 텍스트 제거
                if "yna.co.kr" in content:
                    content = re.sub(r'\S+yna\.co\.kr.*', '', content, flags=re.DOTALL).strip()
                    print("Removed content after 'yna.co.kr'")  # 확인용 로그                
            else:
                content = ""
        else:
            content = ""

        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return "", ""  # 에러 발생 시 빈 문자열 반환

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")
